In [118]:
%run STAGE0_IMPORT_LIBS.ipynb

In [119]:
#data=active_smiles_df+inactive_smiles_df
cwd = os.getcwd()
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
active_data_df=pd.read_csv(parent_dir+'\\0_skdata\chemical_names_pubchem_active.csv')
active_data_df['PubChem_ID'] = active_data_df['PubChem_ID'].astype(str)
inactive_data_df=pd.read_csv(parent_dir+'\\0_skdata\inactive_dude_db_top4.csv')
data = pd.concat([active_data_df, inactive_data_df])
data=data.reset_index(drop=True)
data['data_index'] = data.index

In [120]:
# Load data from CSV file and set probability_factor and corr_factor
probability_factor=0.5
corr_factor=0.2
head_top_smiles=10
testsize=0.2
randomstate=2

In [121]:
input_data=data
selected_cols=['activity', 'SMILES','data_index']
data=data[selected_cols]

In [122]:
##STAGE 5.1: GENERATE DESCRIPTORS, FIND DESCRIPTORS WITH CORRELATION COFFIENT MORE THAN 0.5 AND APPLY ML MODEL

# Convert SMILES strings to molecules
mols = [Chem.MolFromSmiles(smi) for smi in data['SMILES']]
data['Molecule'] = mols

# Calculate molecular descriptors
#X = pd.DataFrame()
data['MW'] = [Descriptors.MolWt(mol) for mol in mols]
data['LogP'] = [Descriptors.MolLogP(mol) for mol in mols]
data['NumHAcceptors'] = [Descriptors.NumHAcceptors(mol) for mol in mols]
data['NumHDonors'] = [Descriptors.NumHDonors(mol) for mol in mols]
data['NumRotatableBonds'] = [Descriptors.NumRotatableBonds(mol) for mol in mols]

data['TPSA'] = [Descriptors.TPSA(mol) for mol in mols]
data['NumAromaticRings'] = [Descriptors.NumAromaticRings(mol) for mol in mols]
data['NumAliphaticRings'] = [Descriptors.NumAliphaticRings(mol) for mol in mols]
data['NumSaturatedRings'] = [Descriptors.NumSaturatedRings(mol) for mol in mols]
data['NumHeteroatoms'] = [Descriptors.NumHeteroatoms(mol) for mol in mols]
data['NumHeavyAtoms'] = [Descriptors.HeavyAtomCount(mol) for mol in mols]
data['NumRings'] = [Descriptors.RingCount(mol) for mol in mols]
data['NumValenceElectrons'] = [Descriptors.NumValenceElectrons(mol) for mol in mols]
data['NumRadicalElectrons'] = [Descriptors.NumRadicalElectrons(mol) for mol in mols]
data['ExactMolWt'] = [Descriptors.ExactMolWt(mol) for mol in mols]
data['FractionCSP3'] = [Descriptors.FractionCSP3(mol) for mol in mols]
X_train=[]
X_test=[]
y_test=[]
y_train=[]
X=[]
y=[]


# Set activity as target variable
y = data['activity']
#X=data
X =  data[['MW', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds','TPSA','NumAromaticRings','NumAliphaticRings','NumSaturatedRings','NumHeteroatoms','NumHeavyAtoms','NumRings','NumValenceElectrons','NumRadicalElectrons','ExactMolWt','FractionCSP3', 'data_index']]
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, stratify=y, random_state=randomstate)
#print(X.shape, X_train.shape, X_test.shape)
selected_features=['MW', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds','TPSA','NumAromaticRings','NumAliphaticRings','NumSaturatedRings','NumHeteroatoms','NumHeavyAtoms','NumRings','NumValenceElectrons','NumRadicalElectrons','ExactMolWt','FractionCSP3']


# Calculate Pearson correlation coefficient for each feature
corr_dict=dict()
for column in X_train[selected_features].columns:
    corr, _ = pearsonr(X_train[column], y_train)
    corr_dict[column] = corr

# Select highly correlated features
highly_corr_features = [feat for feat, corr in corr_dict.items() if abs(corr) > corr_factor]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3668\521994265.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Molecule'] = mols
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3668\521994265.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MW'] = [Descriptors.MolWt(mol) for mol in mols]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3668\521994265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [123]:
print("Selected Correlated Features:",highly_corr_features)

Selected Correlated Features: ['NumAliphaticRings', 'NumSaturatedRings', 'FractionCSP3']


In [124]:
##STAGE 5.2: Apply Machine Learning Model and Predict Probability
nb = GaussianNB()
nb.fit(X_train[highly_corr_features], y_train)
probabilities =       nb.predict_proba(X_test[highly_corr_features])[:, 1]
probabilities_train = nb.predict_proba(X_train[highly_corr_features])[:, 1]

y_pred = (probabilities > probability_factor).astype(int)

accuracy = accuracy_score( y_pred,y_test)

#create an actual column from pseduo column of index which can be used for joining two dataframe

X_test['ProbActivity'] = probabilities
X_test['test_type'] = 'Test Set'

X_train['ProbActivity'] = probabilities_train
X_train['test_type'] = 'Training Set'
print(X.shape, X_train.shape, X_test.shape)

custom_features=['data_index','ProbActivity','test_type']

#custom_features.extend(highly_corr_features)
processed_data=pd.concat([X_test[custom_features], X_train[custom_features]])

(355, 17) (284, 19) (71, 19)


In [125]:
merged_final_data = pd.merge(input_data, processed_data, on='data_index', how='left')
filtered_final_data = merged_final_data.loc[(merged_final_data['activity'] > 0) & (pd.notnull(merged_final_data['ProbActivity'] ))]#.sort_values(by='data_index', ascending=True).head(45)
custom_features.extend(highly_corr_features)
input_data_column=['SMILES']
input_data_column.extend(custom_features)

In [126]:
print("ML Model Accuracy="+str(accuracy))
filtered_final_data.loc[filtered_final_data['Chemical_Name'].isin([ 'Rutoside','Bacoside A3','Diosgenin','Carvacrol','Inulin'])]

ML Model Accuracy=0.7746478873239436


,Chemical_Name,SMILES,PubChem_ID,activity,state,PID,data_index,ProbActivity,test_type
11,Bacoside A3,CC(=CC1CC(C2C3CCC4C5(CCC(C(C5CCC4(C36CC2(O1)OC...,91827005,1,ACTIVE,NaN,11,1.000000,Test Set
22,Carvacrol,CC1=C(C=C(C=C1)C(C)C)O,10364,1,ACTIVE,NaN,22,0.013620,Training Set
28,Diosgenin,CC1CCC2(C(C3C(O2)CC4C3(CCC5C4CC=C6C5(CCC(C6)O)...,99474,1,ACTIVE,NaN,28,1.000000,Training Set
44,Inulin,C(C1C(C(C(C(O1)OC2(C(C(C(O2)CO)O)O)COC3(C(C(C(...,24763,1,ACTIVE,NaN,44,1.000000,Training Set
73,Rutoside,CC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC(...,5280805,1,ACTIVE,NaN,73,0.230652,Test Set
